**=================================================  
BIBLIOTECA DE FUNÇÕES  
=================================================**  

In [1]:
import numpy as np
import pandas as pd

In [2]:
def calculo_nrd_vigas(betaX, **kwargs):
    # Atribuindo valores pelo parâmetro kwargs (Parâmetros da viga)
    betaX2com3 = kwargs.get('betaX2com3')
    defStTrac = kwargs.get('defStTrac')
    defSt50 = kwargs.get('defSt50')
    defSt60 = kwargs.get('defSt60')
    modEAco = kwargs.get('modEAco')
    fydTrac = kwargs.get('fydTrac')
    fydComp = kwargs.get('fydComp')
    d = kwargs.get('d')
    alfaC = kwargs.get('alfaC')
    fcd = kwargs.get('fcd')
    bw = kwargs.get('bw')
    profLambda = kwargs.get('profLambda')
    asTrac = kwargs.get('asTrac')
    asComp = kwargs.get('asComp')
    # Determinação do valor da deformação do aço tracionado
    if betaX <= betaX2com3:
        defStTrac = 10/1000
    else:
        defStTrac = (3.5/1000)/betaX - (3.5/1000)
    # Determinação do valor da tensão do aço tracionado
    if defStTrac < defSt50:
        sigSt = -modEAco*defStTrac
    else:
        sigSt = -fydTrac
    linhaNeutra = betaX*d
    # Determinação do valor da tensão do aço compr. pela tensão do aço trac.
    defStComp = (defStTrac/(linhaNeutra-d))*(linhaNeutra-dlinha)
    # Determinação do valor da tensão do aço comprimido
    if defStComp < defSt60:
        sigStComp = modEAco*defStComp
    else:
        sigStComp = fydComp
    # Força no concreto comprimido
    fRcc = (alfaC*fcd*bw)*(profLambda*linhaNeutra)
    # Força no aço tracionado
    fRst = sigSt*asTrac
    # Força no aço comprimido
    fRsc = sigStComp*asComp
    # Força normal resultante no sistema estrutural
    nRd = fRcc + fRsc + fRst
    return nRd, fRcc, fRsc, fRst 


def metodo_da_bissecao(xMin, xMax, nIter, **kwargs):
    resultados = np.zeros((nIter, 10))
    for iteracao in range(nIter):
        # Cálculo do novo ponto no método da bisseção
        xNew = (xMin + xMax)/2
        # Armazenamento parte 1
        resultados[iteracao][0] = iteracao
        resultados[iteracao][1] = xMin
        resultados[iteracao][2] = xMax
        resultados[iteracao][3] = xNew
        nRdxNew, fRccxNew, fRscxNew, fRstxNew = calculo_nrd_vigas(xNew, **kwargs)
        nRdxMin, fRcc, fRsc, fRst = calculo_nrd_vigas(xMin, **kwargs)
        nRdxMax, fRcc, fRsc, fRst = calculo_nrd_vigas(xMax, **kwargs)
        # Verificação da parcela do domínio onde está registrada a possível
        # raiz da equação
        if (nRdxMin*nRdxNew) <= 0:
            xMin = xMin
            xMax = xNew
        elif (nRdxMin*nRdxNew) > 0:
            xMin = xNew
            xMax = xMax
        resultados[iteracao][4] = xMin
        resultados[iteracao][5] = xMax
        resultados[iteracao][6] = fRccxNew
        resultados[iteracao][7] = fRscxNew
        resultados[iteracao][8] = fRstxNew
        resultados[iteracao][9] = nRdxNew
    return resultados, xNew


**=================================================  
RESOLUÇÃO DO PROBLEMA  
=================================================**  

Aqui são delineadas os dados da viga de concreto conforme exercíco do professor Ricardo Carrazedo

In [3]:
# Dados relativos aos materiais
fck = 30
alfaC = 0.85
profLambda = 0.80
fcd = fck/1.4/10
fykTrac = 50
fykComp = 60
fydTrac = fykTrac/1.15/10
fydComp = fykComp/1.15/10
defSt50 = 2.07/1000
defSt60 = 2.07/1000
modEAco = 21000
asTrac = 5*8
asComp = 2*0.20


# Dados relativos a seção
h = 50
bw = 25
cob = 2.5
fiEstribo = 5
fiLongTrac = 12.5
fiLongComp = 5
 
d = h - (cob+fiEstribo/10+fiLongTrac/2/10);
dlinha = cob+fiEstribo/10+fiLongComp/2/10;
print('d =',d)
print('dlinha =',dlinha)
betaX2com3 = 0.26;
print('Limite Domínio 2 com 3 =', betaX2com3)
betaX3com4 = 0.45;
print('Limite Domínio 3 com 4 =', betaX3com4)

# Dados do método numérico
betaMin = 0.00001
betaMax = d/h
nIter = 50

# Chamada do problema de otimização irrestrita
testeMesa, betaX = metodo_da_bissecao(betaMin, betaMax, nIter, betaX2com3=betaX2com3, defSt50=defSt50, modEAco=modEAco, fydTrac=fydTrac, fydComp=fydComp, d=d, defSt60=defSt60, alfaC=alfaC, fcd=fcd, bw=bw, profLambda=profLambda, asTrac=asTrac, asComp=asComp)
testeMesa = pd.DataFrame(testeMesa)

# Momento resistente
print('betaX =', betaX)
linhaNeutra = betaX*d
print('LN =', linhaNeutra)
# Força no concreto comprimido
fRcc = (alfaC*fcd*bw)*(profLambda*linhaNeutra)
# Força no aço comprimido
fRsc = 50*asComp
Mrd = fRcc*(d-0.50*profLambda*linhaNeutra)+fRsc*(d-dlinha)
print('Mrd =',Mrd)


d = 46.375
dlinha = 3.25
Limite Domínio 2 com 3 = 0.26
Limite Domínio 3 com 4 = 0.45
betaX = 0.10487812795962612
LN = 4.863723184127661
Mrd = 8734.45350738485


In [4]:
testeMesa

,0,1,2,3,4,5,6,7,8,9
0,0.0,0.000010,0.927500,0.463755,0.000010,0.463755,783.456103,-24.957186,-173.913043,5.845859e+02
1,1.0,0.000010,0.463755,0.231882,0.000010,0.231882,391.736498,-17.694347,-173.913043,2.001291e+02
2,2.0,0.000010,0.231882,0.115946,0.000010,0.115946,195.876696,-4.357985,-173.913043,1.760567e+01
3,3.0,0.000010,0.115946,0.057978,0.057978,0.115946,97.946795,1.079200,-173.913043,-7.488705e+01
4,4.0,0.057978,0.115946,0.086962,0.086962,0.115946,146.911746,-1.553092,-173.913043,-2.855439e+01
5,5.0,0.086962,0.115946,0.101454,0.101454,0.115946,171.394221,-2.932919,-173.913043,-5.451742e+00
6,6.0,0.101454,0.115946,0.108700,0.101454,0.108700,183.635458,-3.639659,-173.913043,6.082756e+00
7,7.0,0.101454,0.108700,0.105077,0.101454,0.105077,177.514840,-3.284858,-173.913043,3.169378e-01
8,8.0,0.101454,0.105077,0.103266,0.103266,0.105077,174.454530,-3.108533,-173.913043,-2.567046e+00
9,9.0,0.103266,0.105077,0.104171,0.104171,0.105077,175.984685,-3.196607,-173.913043,-1.124965e+00
